In [ ]:
import pandas as pd 
import numpy  as np
import json

file_path = '/home/mantovani/Documents/IC/HMC_TE-s/Temp_Data/Dados_Dominios_Conservados/sequences.json'

df:dict = {'Name': []}

with open(file_path, mode='r') as file:

    dados = json.load(file)

    for i in dados['results']:

        df['Name'].append(i['crossReferences'][0]['name'].strip()[0:11])

        l_reading_frames, l_df = len(i['openReadingFrames']), (len(df)-1)//4

        print(i['openReadingFrames'],'\n', len(i['openReadingFrames']))

        for j in range(max(l_reading_frames, l_df)):

            reading_frames = i['openReadingFrames'][j]

            if j > l_df or l_df == 0:

                l = [np.nan for _ in range(len(df['Name'])-1)]#pois o nome é gerado antes

                df['Start_{}'.format(j)]             = l
                df['End_{}'.format(j)]               = l
                df['Strand_{}'.format(j)]            = l
                df['Protein_Sequence_{}'.format(j)]  = l

            elif j <= l_reading_frames:

                df['Start_{}'.format(j)].append(reading_frames['start'])
                df['End_{}'.format(j)].append(reading_frames['end'])
                df['Strand_{}'.format(j)].append(reading_frames['strand'])
                df['Protein_Sequence_{}'.format(j)].append(reading_frames['protein']['sequence'])

            else:

                df['Start_{}'.format(j)].append(np.nan)
                df['End_{}'.format(j)].append(np.nan)
                df['Strand_{}'.format(j)].append(np.nan)
                df['Protein_Sequence_{}'.format(j)].append(np.nan)

    file.close()

In [47]:
Aminoacido_Nucleotidio = {
    'F': ('UUU', 'UUC'),
    'L': ('UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'),
    'I': ('AUU', 'AUC', 'AUA'),
    'M': ('AUG'),
    'V': ('GUU', 'GUC', 'GUA', 'GUG'),
    'S': ('UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'),
    'P': ('CCU', 'CCC', 'CCA', 'CCG'),
    'T': ('ACU', 'ACC', 'ACA', 'ACG'),
    'A': ('GCU', 'GCC', 'GCA', 'GCG'),
    'Y': ('UAU', 'UAC'),
    'H': ('CAU', 'CAC'),
    'Q': ('CAA', 'CAG'),
    'N': ('AAU', 'AAC'),
    'K': ('AAA', 'AAG'),
    'D': ('GAU', 'GAC'),
    'E': ('GAA', 'GAG'),
    'C': ('UGU', 'UGC'),
    'W': ('UGG'),
    'R': ('CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'),
    'G': ('GGU', 'GGC', 'GGA', 'GGG'),
    '*': ('UAA', 'UAG', 'UGA')  # Códons de parada
}
    

def Proteina_trinca(aminoacido: str) -> np.ndarray:

    def grafo_N(nulceotidio: str) -> list:

        nulceotidio:str = nulceotidio.upper()

        if nulceotidio == 'A':

            return [1,0,0,0]
        
        elif nulceotidio == 'C':

            return [0,1,0,0]
        
        elif nulceotidio == 'G':

            return [0,0,1,0]
        
        elif nulceotidio == 'U':

            return [0,0,0,1]
        
        else: 

            raise KeyError

    aminoacido = aminoacido.upper()
    
    A = None

    for i in Aminoacido_Nucleotidio[aminoacido]:
    
        codon_array = np.array([grafo_N(n) for n in i])

        if A is None:

            A = codon_array

        else:

            A = np.logical_or(A, codon_array)

    return A.astype(int) 

print(Proteina_trinca('S'))

[[1 0 0 1]
 [0 1 1 0]
 [1 1 1 1]]


In [42]:
import numpy as np

print(np.array(((0, 0, 0, 1), (0, 1, 0, 0), (0, 0, 0, 1))))

[[0 0 0 1]
 [0 1 0 0]
 [0 0 0 1]]
